In [1]:
import gradio as gr
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score

# Load the dataset
crime = pd.read_csv('05. crime_dataset_analysis.csv')
crime = pd.DataFrame(crime)

# Dummy model setup for demonstration
X_dummy = crime.drop(columns=['Weapon Category'])
y_dummy = crime['Weapon Category']
model = LogisticRegression()
model.fit(X_dummy, y_dummy)

# Helper function to convert "Yes"/"No" to numeric
def convert_input(value):
    return 1 if value == "Yes" else 0

# Function to format confusion matrix with labels
def format_confusion_matrix(matrix, labels):
    return f"{labels[0]}\\{labels[1]:>15} {labels[1]} \n" + \
           f"{labels[0]}{matrix[0][0]:15} {matrix[0][1]:15} \n" + \
           f"{labels[1]}{matrix[1][0]:15} {matrix[1][1]:15}"

# Gradio interface function
def gradio_interface(
    region_northeast, region_south, region_west,
    season_spring, season_summer, season_winter,
    relationship_family, relationship_lover, relationship_stranger,
    agency_other, agency_sheriff,
    victim_sex_male, perpetrator_sex_male,
    ethnicity_not_hispanic, victim_age_scaled
):
    # Convert dropdown inputs ("Yes"/"No") to numeric
    input_features = [
        convert_input(region_northeast), convert_input(region_south), convert_input(region_west),
        convert_input(season_spring), convert_input(season_summer), convert_input(season_winter),
        convert_input(relationship_family), convert_input(relationship_lover), convert_input(relationship_stranger),
        convert_input(agency_other), convert_input(agency_sheriff),
        convert_input(victim_sex_male), convert_input(perpetrator_sex_male),
        convert_input(ethnicity_not_hispanic),
        victim_age_scaled,  # Victim age remains numeric
    ]

    # Model predictions
    input_pred = model.predict([input_features])[0]  # Single prediction
    input_pred_proba = model.predict_proba([input_features])[0]  # Probabilities for both classes

    # Map numeric prediction to labels
    prediction_label = "Firearm" if input_pred == 1 else "Non-Firearm"

    # Compute evaluation metrics
    y_pred = model.predict(X_dummy)
    conf_matrix = confusion_matrix(y_dummy, y_pred)
    accuracy = accuracy_score(y_dummy, y_pred)
    recall = recall_score(y_dummy, y_pred)
    precision = precision_score(y_dummy, y_pred)
    f1 = f1_score(y_dummy, y_pred)

    # Format confusion matrix with labels
    matrix_labels = ["Non-Firearm", "Firearm"]
    formatted_conf_matrix = format_confusion_matrix(conf_matrix, matrix_labels)

    return (
        f"Prediction: {prediction_label}",
        f"Probability of Firearm Being Used: {input_pred_proba[1]:.2f}",
        f"Probability of Firearm Not Being Used: {input_pred_proba[0]:.2f}",
        f"Confusion Matrix:\n{formatted_conf_matrix}",
        f"Accuracy: {accuracy:.2f}",
        f"Recall: {recall:.2f}",
        f"Precision: {precision:.2f}",
        f"F1 Score: {f1:.2f}"
    )

# Gradio dropdowns for inputs
input_features = [
    gr.Dropdown(["No", "Yes"], value="No", label="Region_Northeast"),
    gr.Dropdown(["No", "Yes"], value="No", label="Region_South"),
    gr.Dropdown(["No", "Yes"], value="No", label="Region_West"),
    gr.Dropdown(["No", "Yes"], value="No", label="Season_Spring"),
    gr.Dropdown(["No", "Yes"], value="No", label="Season_Summer"),
    gr.Dropdown(["No", "Yes"], value="No", label="Season_Winter"),
    gr.Dropdown(["No", "Yes"], value="No", label="Relationship Category_Family"),
    gr.Dropdown(["No", "Yes"], value="No", label="Relationship Category_Lover"),
    gr.Dropdown(["No", "Yes"], value="No", label="Relationship Category_Stranger"),
    gr.Dropdown(["No", "Yes"], value="No", label="Agency_Type_grouped_Other Police"),
    gr.Dropdown(["No", "Yes"], value="No", label="Agency_Type_grouped_Sheriff"),
    gr.Dropdown(["No", "Yes"], value="No", label="Victim Sex_Male"),
    gr.Dropdown(["No", "Yes"], value="No", label="Perpetrator Sex_Male"),
    gr.Dropdown(["No", "Yes"], value="No", label="Ethnicity_Not_Hispanic_Combined"),
    gr.Slider(-3, 3, step=0.1, value=0, label="Victim Age Scaled"),
]

# Custom CSS for the Gradio interface
custom_css = """
body {
    background: linear-gradient(to bottom right, #6a11cb, #2575fc); /* Gradient background */
    color: white;
    font-family: Arial, sans-serif;
}
.gradio-container {
    background: rgba(180, 180, 180, 0.5); /* Semi-transparent white */
    border-radius: 15px;
    padding: 20px;
    box-shadow: 0px 4px 12px rgba(0, 0, 0, 0.2);
}
.gradio-container .input-block input, 
.gradio-container .input-block select, 
.gradio-container .input-block textarea {
    background: #e0f7fa; /* Light cyan for input boxes */
    border: 1px solid #00796b;
    border-radius: 5px;
    padding: 5px;
    color: #004d40;
}
.gradio-container .output-block textarea {
    background: #ffebee; /* Light pink for output boxes */
    border: 1px solid #d32f2f;
    border-radius: 5px;
    padding: 5px;
    color: #b71c1c;
}
"""


# Gradio interface
gr.Interface(
    title="Weapon Use Prediction",
    description="**Instructions:** For each category (Region, Season, Relationship, Agency), only one 'Yes' value is allowed. All other options should be 'No'. You can leave them all as 'No' if necessary.", 
    fn=gradio_interface,
    inputs=input_features,
    outputs=[
        gr.Textbox(label="Prediction"),
        gr.Textbox(label="Probability of Firearm Being Used"),
        gr.Textbox(label="Probability of Firearm Not Being Used"),
        gr.Textbox(label="Confusion Matrix"),
        gr.Textbox(label="Accuracy"),
        gr.Textbox(label="Recall"),
        gr.Textbox(label="Precision"),
        gr.Textbox(label="F1 Score"),
    ],
    css=custom_css,
).launch()


* Running on local URL:  http://127.0.0.1:7881

To create a public link, set `share=True` in `launch()`.
